# Chaning total Pressure and analysing blood flow in each vessel

In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

## Baseline Conditions

#### For Attention List:
1. Need to go through and check the units work properly. Note that the values didnt change when I removed units so seems okay
2. Check units on flows
3. Check units on Vt
4. Add units to to all columns and decide upon my convention for this
5. Check units and speak to Payne about the wall thickness calcs
6. Is velocity redundant?

#### Set Constants and paramaters

In [68]:
#input params
H = 0.42 #no units,ratio #Hematocrit assumed to be constant
M = 30* 10e-9 #mol_ O2/(mL/s) #Taken from table 2 from Wiley Payne paper
cHb = 0.2 #mL_O2/m #Taken from table 2 from Wiley Payne paper
paO2_bar_t = 15 #mmHG #Taken from table 2 from Wiley Payne paper
K = 5*10e-8

##reset df
df = None

#Input data from paper
df = pd.DataFrame({'Name': ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'C', 'V6', 'V5', 'V4', 'V3', 'V2', 'V1'], 
                   'Number': [1, 2, 4, 8, 16, 32, 64, 32, 16, 8, 4, 2, 1],
                   'Diameter(µm)': [23.97, 19.17, 15.28, 12.08, 9.46, 7.32, 8, 11.51, 14.53, 17.79, 21.45, 25.70, 30.77],
                   'Length(µm)': [1267.6, 930.3, 543.6, 302.3, 161.2, 154.7, 243.9, 473.9, 272.3, 426.6, 632.5, 844.2, 936.3],
                   'Viscocity(mPAs)': [1.59, 1.50, 1.42, 1.34, 1.28, 1.23, 1.24, 1.33, 1.40, 1.48, 1.55, 1.62, 1.7],
                   'Pressure Drop(mmHg)': [6.93, 5.87, 4.02, 2.70, 1.82, 2.35, 2.62, 1.27, 0.61, 0.89, 1.31, 1.78, 2.01],
                   'Saturation': [0.94, 0.93, 0.92, 0.89, 0.84, 0.765, 0.665, 0.61, 0.5975, 0.5875, 0.5825, 0.5775, 0.5725],
                   'dS': [0.01, 0.01, 0.02, 0.04, 0.0625,0.0875, 0.0775, 0.03375, 0.01125, 0.0075, 0.005, 0.005, 0.005]
                  })


#### Calculate Flows and Vt

In [69]:
#Calculate resistances of each vessel
df['Resistance for U'] = 32 * df['Viscocity(mPAs)'] * df['Length(µm)'].div(df['Diameter(µm)']**2)
df['Resistance for Q'] = 128 * df['Viscocity(mPAs)'] * df['Length(µm)'].div(math.pi*(df['Diameter(µm)']**4))

#Alter resistances for C
#These are aross each pressure drop (explains the doubled resistance for the capillaries as in paralell so have half the flow each (flow is shared))
df.loc[df.index[df['Name'] == 'C'][0],'Resistance for U'] = df.loc[df.index[df['Name'] == 'C'][0],'Resistance for U']*2
df.loc[df.index[df['Name'] == 'C'][0],'Resistance for Q'] = df.loc[df.index[df['Name'] == 'C'][0],'Resistance for Q']*2

#Calculate flow and velocity in a single vessel
df['U in single(m/s????)'] = df['Pressure Drop(mmHg)'].div(df['Resistance for U'])
df['Q in single(m^3/s????)'] = df['Pressure Drop(mmHg)'].div(df['Resistance for Q'])

#Find change in saturdation from data on saturation curve
# S = pd.Series([])

#Calculate Vt CHANGE IN SATURATIO  YA SHIT HEAD
df['Vt'] = df['Q in single(m^3/s????)']*cHb*H*df['dS']/M

#### Calculate partial pressure in blood
S and partial O2 relationship given by servinghaus 1979  
Checked partial pressure against typical values online and it seems bang on.

In [70]:
def check_imag_roots_real(all_roots):
    counter = 0
    sols = 0
    for i in range(len(all_roots)):
        if all_roots[i].imag <= 10e-13 and all_roots[i].imag >= -10e-13:
            sols = all_roots[i].real
            counter += 1
    if counter != 1:
        print('Something seems wrong with the partial presure to Saturation cubic solver as not one solution is returned.')
    return sols
 
for i in range(len(df['Saturation'])):
    root_temp = None
    sols = 0
    root_temp = np.roots([1,0,150,23400*df.loc[i,'Saturation']/(df.loc[i,'Saturation']-1)]) #gives mmHg , *133.322
    sols = check_imag_roots_real(root_temp) 
    #sols = sols*133.322 ###Remove this step if want in mmHg
    df.loc[i,'partial pressure blood(mmHg)'] = sols

#### Calculate wall thickness
Will take my vessel average partial pressure to be the 0.5(pin + pout)  
All units were in mm so i played around with this equation from the paper  
Equations are taken from Payne paper

In [71]:
#all units were in mm so i played around with this equation from the paper
df['X-Area wall(mm^2?)'] = np.pi * ( 0.16*(df['Diameter(µm)'])**2 + 1.4*(df['Diameter(µm)']) + 14 )
df['wall thickness(µm)'] = ( -(df['Diameter(µm)']) + np.sqrt((df['Diameter(µm)'])**2 + (4*(df['X-Area wall(mm^2?)'])/np.pi)) ) / 2

#### Calculate partial pressure in tissue
Equations can be found on the week 1 or 2 notes from class. From steady state version of something on the paper

In [72]:
df['tissue partials(mmHg)'] = df['partial pressure blood(mmHg)'] - M * df['Vt'] * df['wall thickness(µm)'] / (2 * np.pi * K * (df['Diameter(µm)']/2) * df['Length(µm)'])

#### Pull out the baseline conditions

In [73]:
df_base = df
df_base

,Name,Number,Diameter(µm),Length(µm),Viscocity(mPAs),Pressure Drop(mmHg),Saturation,dS,Resistance for U,Resistance for Q,U in single(m/s????),Q in single(m^3/s????),Vt,partial pressure blood(mmHg),X-Area wall(mm^2?),wall thickness(µm),tissue partials(mmHg)
0,A1,1,23.97,1267.6,1.59,6.93,0.9400,0.01000,112.251787,0.248752,0.061736,27.859025,78005.271056,70.871373,438.213672,4.841407,68.497559
1,A2,2,19.17,930.3,1.50,5.87,0.9300,0.01000,121.512242,0.421004,0.048308,13.942870,39040.035790,67.005340,313.016429,4.253658,65.226941
2,A3,4,15.28,543.6,1.42,4.02,0.9200,0.02000,105.796442,0.576945,0.037997,6.967734,39019.312012,63.786724,228.546291,3.810683,60.367867
3,A4,8,12.08,302.3,1.34,2.70,0.8900,0.04000,88.829876,0.775060,0.030395,3.483601,39016.335895,56.550348,170.463521,3.485850,49.437369
4,A5,16,9.46,161.2,1.28,1.82,0.8400,0.06250,73.780690,1.049713,0.024668,1.733807,30341.627914,48.706014,130.572935,3.265971,36.295309
5,A6,32,7.32,154.7,1.23,2.35,0.7650,0.08750,113.637911,2.700295,0.020680,0.870275,21321.745183,41.211431,103.110791,3.138293,29.926051
6,C,64,8.00,243.9,1.24,2.62,0.6650,0.07750,302.436000,6.016773,0.008663,0.435449,9449.251212,34.556920,111.338044,3.172168,31.622967
7,V6,32,11.51,473.9,1.33,1.27,0.6100,0.03375,152.243122,1.463178,0.008342,0.867974,8202.351518,31.696836,161.197683,3.433626,30.710714
8,V5,16,14.53,272.3,1.40,0.61,0.5975,0.01125,57.782247,0.348476,0.010557,1.750477,5514.003939,31.097186,214.009514,3.730525,30.104242
9,V4,8,17.79,426.6,1.48,0.89,0.5875,0.00750,63.838202,0.256826,0.013941,3.465382,7277.302540,30.628584,281.309064,4.092089,29.879173


## Altering p_tot

Above is the normal model. Now i will take whatever parts are useful and run again for my new pressure drops.

#### For Attention List:
1. Can i imporve the resistance calculating bit?
2. 

#### Find Resistance of baseline model

In [78]:
baselines = df_base[['Name','Number','Resistance for Q','Q in single(m^3/s????)','Vt','wall thickness(µm)','tissue partials(mmHg)','Diameter(µm)','Length(µm)']].rename(columns={"Q in single(m^3/s????)": "baseline Q", "tissue partials(mmHg)": "baseline tissue partial"})

In [79]:
#Find Total Resistance. C refers to capilaary and each numer refers to an added level or resistance.
C_ = baselines.loc[6,'Resistance for Q']/2
C_6 = (C_ + baselines.loc[5,'Resistance for Q'] + baselines.loc[7,'Resistance for Q'])/2
C_65 = (C_6 + baselines.loc[4,'Resistance for Q'] + baselines.loc[8,'Resistance for Q'])/2
C_654 = (C_65 + baselines.loc[3,'Resistance for Q'] + baselines.loc[9,'Resistance for Q'])/2
C_6543 = (C_654 + baselines.loc[2,'Resistance for Q'] + baselines.loc[10,'Resistance for Q'])/2
C_65432 = (C_6543 + baselines.loc[1,'Resistance for Q'] + baselines.loc[11,'Resistance for Q'])/2
C_654321 = C_65432 + baselines.loc[0,'Resistance for Q'] + baselines.loc[12,'Resistance for Q']
R_total = C_654321

#Proves for the baseline conditions that it gives the right total flow from my resitance calcs.
#34.18 is pressure drop
Q_total = 34.18 / C_654321
print(Q_total)

27.848258609370426


In [80]:
saturation_in = 0.945
######## run_0 ######
#assume 
pressure_in = (60-34.18/2)
pressure_out = 60 - 34.18
pressure_difference = pressure_in - pressure_out

Q_in = pressure_difference/R_total
run_0 = pd.DataFrame({'Name': ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'C', 'V6', 'V5', 'V4', 'V3',
       'V2', 'V1']})
run_0['Q'] = np.empty
run_0.loc[0,'Q'] = Q_in
run_0.loc[1,'Q'] = run_0.loc[0,'Q']/2
run_0.loc[2,'Q'] = run_0.loc[1,'Q']/2
run_0.loc[3,'Q'] = run_0.loc[2,'Q']/2
run_0.loc[4,'Q'] = run_0.loc[3,'Q']/2
run_0.loc[5,'Q'] = run_0.loc[4,'Q']/2
run_0.loc[6,'Q'] = run_0.loc[5,'Q']/2
run_0.loc[7,'Q'] = run_0.loc[6,'Q']*2
run_0.loc[8,'Q'] = run_0.loc[7,'Q']*2
run_0.loc[9,'Q'] = run_0.loc[8,'Q']*2
run_0.loc[10,'Q'] = run_0.loc[9,'Q']*2
run_0.loc[11,'Q'] = run_0.loc[10,'Q']*2
run_0.loc[12,'Q'] = run_0.loc[11,'Q']*2

run_0['dS'] = ( baselines['Vt'] * M ) / ( cHb * H * run_0['Q'] )
run_0['Saturation'] = np.empty

#due to the 0.945 being from an inital value above.
run_0.loc[0,'Saturation'] = saturation_in - 0.5*run_0.loc[0,'dS']
run_0.loc[1,'Saturation'] = run_0.loc[0,'Saturation'] - 0.5*run_0.loc[0,'dS'] - 0.5*run_0.loc[1,'dS']
run_0.loc[2,'Saturation'] = run_0.loc[1,'Saturation'] - 0.5*run_0.loc[1,'dS'] - 0.5*run_0.loc[2,'dS']
run_0.loc[3,'Saturation'] = run_0.loc[2,'Saturation'] - 0.5*run_0.loc[2,'dS'] - 0.5*run_0.loc[3,'dS']
run_0.loc[4,'Saturation'] = run_0.loc[3,'Saturation'] - 0.5*run_0.loc[3,'dS'] - 0.5*run_0.loc[4,'dS']
run_0.loc[5,'Saturation'] = run_0.loc[4,'Saturation'] - 0.5*run_0.loc[4,'dS'] - 0.5*run_0.loc[5,'dS']
run_0.loc[6,'Saturation'] = run_0.loc[5,'Saturation'] - 0.5*run_0.loc[5,'dS'] - 0.5*run_0.loc[6,'dS']
run_0.loc[7,'Saturation'] = run_0.loc[6,'Saturation'] - 0.5*run_0.loc[6,'dS'] - 0.5*run_0.loc[7,'dS']
run_0.loc[8,'Saturation'] = run_0.loc[7,'Saturation'] - 0.5*run_0.loc[7,'dS'] - 0.5*run_0.loc[8,'dS']
run_0.loc[9,'Saturation'] = run_0.loc[8,'Saturation'] - 0.5*run_0.loc[8,'dS'] - 0.5*run_0.loc[9,'dS']
run_0.loc[10,'Saturation'] = run_0.loc[9,'Saturation'] - 0.5*run_0.loc[9,'dS'] - 0.5*run_0.loc[10,'dS']
run_0.loc[11,'Saturation'] = run_0.loc[10,'Saturation'] - 0.5*run_0.loc[10,'dS'] - 0.5*run_0.loc[11,'dS']
run_0.loc[12,'Saturation'] = run_0.loc[11,'Saturation'] - 0.5*run_0.loc[11,'dS'] - 0.5*run_0.loc[12,'dS']

for i in range(len(run_0['Saturation'])):
    root_temp = None
    sols = 0
    root_temp = np.roots([1,0,150,23400*run_0.loc[i,'Saturation']/(run_0.loc[i,'Saturation']-1)]) #gives mmHg , *133.322
    sols = check_imag_roots_real(root_temp) 
    #sols = sols*133.322 ###Remove this step if want in mmHg
    run_0.loc[i,'partial pressure blood(mmHg)'] = sols
    
run_0['tissue partials(mmHg)'] = run_0['partial pressure blood(mmHg)'] - M * baselines['Vt'] * baselines['wall thickness(µm)'] / (2 * np.pi * K * (baselines['Diameter(µm)']/2) * baselines['Length(µm)'])

In [81]:
run_0

,Name,Q,dS,Saturation,partial pressure blood(mmHg),tissue partials(mmHg)
0,A1,13.9241,0.0200077,0.934996,68.841641,66.467827
1,A2,6.96206,0.0200269,0.914979,62.357277,60.578879
2,A3,3.48103,0.0400326,0.884949,55.575431,52.156573
3,A4,1.74052,0.080059,0.824903,46.906474,39.793495
4,A5,0.870258,0.124518,0.722615,38.086195,25.675491
5,A6,0.435129,0.175003,0.572854,29.958606,18.673226
6,C,0.217565,0.155114,0.407795,23.282564,20.348611
7,V6,0.435129,0.0673228,0.296577,19.126494,18.140372
8,V5,0.870258,0.0226288,0.251601,17.389327,16.396382
9,V4,1.74052,0.0149326,0.23282,16.637836,15.888425


In [ ]:
df_base_useful = df_base[['Name',
        'Resistance for Q',  'Vt']]

In [ ]:
#Find Total Resistance. C refers to capilaary and each numer refers to an added level or resistance.
C_ = df_base_useful.loc[6,'Resistance for Q']/2
C_6 = (C_ + df_base_useful.loc[5,'Resistance for Q'] + df_base_useful.loc[7,'Resistance for Q'])/2
C_65 = (C_6 + df_base_useful.loc[4,'Resistance for Q'] + df_base_useful.loc[8,'Resistance for Q'])/2
C_654 = (C_65 + df_base_useful.loc[3,'Resistance for Q'] + df_base_useful.loc[9,'Resistance for Q'])/2
C_6543 = (C_654 + df_base_useful.loc[2,'Resistance for Q'] + df_base_useful.loc[10,'Resistance for Q'])/2
C_65432 = (C_6543 + df_base_useful.loc[1,'Resistance for Q'] + df_base_useful.loc[11,'Resistance for Q'])/2
C_654321 = C_65432 + df_base_useful.loc[0,'Resistance for Q'] + df_base_useful.loc[12,'Resistance for Q']
R_total = C_654321

#Proves for the baseline conditions that it gives the right total flow from my resitance calcs.
#34.18 is pressure drop
Q_total = 34.18 / C_654321
print(Q_total)

#### Calculates Total flow for each pressure drop.

In [ ]:
pressure_out = 60 - 34.18 # a constant in my model

no_steps = 100

#we vary by plus and minus 50%
main = pd.DataFrame({'Pressure in(mmHg)': np.linspace((60-34.18/2),(60+34.18/2), num=no_steps), 'Pressure out(mmHg)': np.zeros(no_steps) + pressure_out})
main['pressure drop (mmHg)'] = main['Pressure in(mmHg)'] - main['Pressure out(mmHg)']
main['Total Flow'] = main['pressure drop (mmHg)']/R_total

def add_vessel_flow():
    C = pd.DataFrame({'Flow': main['Total Flow'] / df.loc[6,'Number']})
    
    return vessel

#Do for C first
C = pd.DataFrame({'Flow': main['Total Flow'] / df.loc[6,'Number']})
C['dS'] = df.loc[6,'Vt'] * M / (cHb * H * C['Flow'])

C

In [ ]:
main.head(5)

In [ ]:
multiple_dataframes = pd.DataFrame([main,C])

In [ ]:
multiple_dataframes.iloc[0,0]

In [ ]:
df

In [77]:
baselines

,Name,Number,Resistance for Q,baseline Q,Vt,wall thickness(µm),baseline tissue partial
0,A1,1,0.248752,27.859025,78005.271056,4.841407,68.497559
1,A2,2,0.421004,13.942870,39040.035790,4.253658,65.226941
2,A3,4,0.576945,6.967734,39019.312012,3.810683,60.367867
3,A4,8,0.775060,3.483601,39016.335895,3.485850,49.437369
4,A5,16,1.049713,1.733807,30341.627914,3.265971,36.295309
5,A6,32,2.700295,0.870275,21321.745183,3.138293,29.926051
6,C,64,6.016773,0.435449,9449.251212,3.172168,31.622967
7,V6,32,1.463178,0.867974,8202.351518,3.433626,30.710714
8,V5,16,0.348476,1.750477,5514.003939,3.730525,30.104242
9,V4,8,0.256826,3.465382,7277.302540,4.092089,29.879173
